# Desagregação dos dados da bacia do Parnaíba usando GeoPandas

Este *notebook* mostra é uma continuação [deste *notebook*](https://github.com/francisconog/PBCE/blob/master/analysis/ESTPLU_POR_BACIA.ipynb) que classifica um ponto de acordo com sua latitude e longitude. Em 2012 a Bacia do Parnaíba foi dividida em: Serra de Ibiapaba e Sertões de Crateús. Desta forma, precisa-se desagregar os dados da antiga bacia para a realização de análises mais completas.

*Este Notebook faz parte de uma colaboração da Universidade Federal do Ceará (UFC), da Companhia de Gestão de Recursos Hídricos (COGERH) e da Fundação Cearense de Apoio ao Desenvolvimento Científico e Tecnológico (FUNCAP) no âmbito do projeto de desenvolvimento de ferramentas tecnológicas de gestão para planejamento de recursos hídricos do estado do Ceará: Segurança hídrica e planejamento de secas*

*Code by: Francisco Nogueira*

## Importing packages

In [1]:
import geobr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os
from shapely.geometry import Point
from tqdm import tqdm
import sys
import warnings
sys.path.append('../src')
warnings.filterwarnings('ignore')
from functions import *
import re

## Paths

In [2]:
data_path = "../data/"
shape_file_path = "shapes"
shape_malha_dagua = "Drenagem/Margens_simples_CE_corrigidoSRH.shp"
shape_bacias = "geomap_bacias/i3geomap_bacias.shp"
faturamento_data_path = "Dados Alocar/faturamento-cogerh"

## Import Data

In [3]:
# Dados de outorga de 2012
faturamento_2012 = pd.read_excel(os.path.join(data_path,faturamento_data_path,"historico_faturamento.xlsx"))
faturamento_2012.head()


,Gerencia,Bacia,Infra Estrutura Hídrica,Municipio,Categoria,Consumo m³,Mês,Ano
0,GERENCIA METROPOLITANA,BACIA METROPOLITANA - GERENCIA,CANAL DO TRABALHADOR,FORTALEZA,A.BRUTA CAGECE INTERIOR,9599.37,maio,2012
1,GERENCIA METROPOLITANA,BACIA METROPOLITANA - GERENCIA,CANAL DO TRABALHADOR,FORTALEZA,A.BRUTA CAGECE INTERIOR,1008.50,junho,2012
2,GERENCIA METROPOLITANA,BACIA METROPOLITANA - GERENCIA,CANAL DO TRABALHADOR,FORTALEZA,A.BRUTA CAGECE INTERIOR,1118.50,julho,2012
3,GERENCIA METROPOLITANA,BACIA METROPOLITANA - GERENCIA,CANAL DO TRABALHADOR,FORTALEZA,A.BRUTA CAGECE INTERIOR,1322.00,agosto,2012
4,GERENCIA METROPOLITANA,BACIA METROPOLITANA - GERENCIA,CANAL DO TRABALHADOR,FORTALEZA,A.BRUTA CAGECE INTERIOR,1628.50,setembro,2012


##### Agora que carregados os pacotes e os dados, vamos ao que interessa...

In [4]:
lista_bacias = faturamento_2012["Bacia"].unique()
update_lista_bacias = {x: x.split(" -")[0] for x in lista_bacias}


In [5]:
faturamento_2012["Bacia"] = faturamento_2012["Bacia"].apply(lambda x: update_lista_bacias[x])

faturamento_2012["Bacia"].unique()

array(['BACIA METROPOLITANA', 'BACIA DO CURU',
       'BACIA DO MEDIO JAGUARIBE ', 'BACIA DO SALGADO',
       'BACIA DO ALTO JAGUARIBE', 'BACIA DO LITORAL ', 'BACIA DO ACARAU ',
       'BACIA DO CURU E LITORAL ', 'BACIA DO BAIXO JAGUARIBE',
       'BACIA DO BAIXO E MEDIO JAGUARI', 'BACIA DO COREAU',
       'BACIA DO PARNAIBA', 'BACIA DO ACARAU E COREAU',
       'BACIA DO BANABUIU'], dtype=object)

In [6]:
# Dados de faturamento referentes à bacia do Parnaíba
faturamento_parnaiba = faturamento_2012.query("Bacia=='BACIA DO PARNAIBA'")
faturamento_parnaiba.head()

,Gerencia,Bacia,Infra Estrutura Hídrica,Municipio,Categoria,Consumo m³,Mês,Ano
118664,GERENCIA DE SOBRAL,BACIA DO PARNAIBA,ACUDE JABURU I,MUCAMBO,A.BRUTA SAAES E PREFEITURAS,38112.0,janeiro,2007
118665,GERENCIA DE SOBRAL,BACIA DO PARNAIBA,ACUDE JABURU I,MUCAMBO,A.BRUTA SAAES E PREFEITURAS,38691.0,fevereiro,2007
118666,GERENCIA DE SOBRAL,BACIA DO PARNAIBA,ACUDE JABURU I,MUCAMBO,A.BRUTA SAAES E PREFEITURAS,36996.0,março,2007
118667,GERENCIA DE SOBRAL,BACIA DO PARNAIBA,ACUDE JABURU I,MUCAMBO,A.BRUTA SAAES E PREFEITURAS,28571.0,abril,2007
118668,GERENCIA DE SOBRAL,BACIA DO PARNAIBA,ACUDE JABURU I,MUCAMBO,A.BRUTA SAAES E PREFEITURAS,31906.0,maio,2007


In [7]:
faturamento_parnaiba.query("`Infra Estrutura Hídrica`=='BACIA PARNAIBA'")

,Gerencia,Bacia,Infra Estrutura Hídrica,Municipio,Categoria,Consumo m³,Mês,Ano
121088,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,SAO BENEDITO,A. BRUTA SERV E COM AD PROPRIA,488.00,janeiro,2018
121089,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,SAO BENEDITO,A. BRUTA SERV E COM AD PROPRIA,360.00,abril,2018
121090,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,SAO BENEDITO,A. BRUTA SERV E COM AD PROPRIA,364.00,julho,2018
121091,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,SAO BENEDITO,A. BRUTA SERV E COM AD PROPRIA,368.00,outubro,2018
121092,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,SAO BENEDITO,A. BRUTA SERV E COM AD PROPRIA,368.00,janeiro,2019
...,...,...,...,...,...,...,...,...
121307,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,GUARACIABA DO N,A.BRUTA DEMAIS CATEGORIAS,315.48,março,2017
121308,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,GUARACIABA DO N,A.BRUTA DEMAIS CATEGORIAS,366.00,maio,2017
121309,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,GUARACIABA DO N,A.BRUTA DEMAIS CATEGORIAS,366.00,julho,2017
121310,GERENCIA DE CRATEUS,BACIA DO PARNAIBA,BACIA PARNAIBA,GUARACIABA DO N,A.BRUTA DEMAIS CATEGORIAS,372.00,setembro,2017


In [8]:
# A desagregação dos Dados será feita baseado na Gerencia
dict_gerencia_bacia = {
    'GERENCIA DE SOBRAL': "BACIA DA SERRA DA IBIAPABA",
    'GERENCIA DE CRATEUS': 'BACIA DO SERTÕES DE CRATEÚS',
    'GERENCIA DE QUIXERAMOBIM': 'BACIA DO BANABUIU'
}

def atualizacao_bacia(df):
    try:
        return dict_gerencia_bacia[df["Gerencia"]]
    except:
        return df["Bacia"]

In [9]:
faturamento_2012["Bacia"] = faturamento_2012.apply(atualizacao_bacia,axis=1)

In [10]:
# Agora é só salvar salvar os dados de faturamento desagregados
faturamento_2012.to_excel(os.path.join(data_path,faturamento_data_path,"faturamento_limpo_desagregado.xlsx"),index=False)